### mlp mixer

In [9]:
import timm
import torch.nn as nn

mixer = timm.create_model('mixer_b16_224', pretrained=True)
num_classes = 3  # 새로운 클래스 수
mixer.head = nn.Linear(mixer.head.in_features, num_classes)
mixer.eval()


MlpMixer(
  (stem): PatchEmbed(
    (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    (norm): Identity()
  )
  (blocks): Sequential(
    (0): MixerBlock(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_tokens): Mlp(
        (fc1): Linear(in_features=196, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=384, out_features=196, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp_channels): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (drop1): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop2): Dropout(p=0.0, inplace=False)
      

In [10]:
import torch
import torch.nn as nn
checkpoint = torch.load("C:/Users/chldy/Downloads/yong_mlp_checkpoint.bin", map_location='cpu') # GPU 환경이 아닌 경우 'cpu'를 사용합니다.
mixer.load_state_dict(checkpoint)



<All keys matched successfully>

### vit 

In [11]:
import timm
vit = timm.create_model('vit_base_patch16_224', pretrained=True)

num_classes = 3  # 새로운 클래스 수
vit.head = nn.Linear(vit.head.in_features, num_classes)


In [12]:
import torch
import torch.nn as nn
checkpoint = torch.load("C:/Users/chldy/Downloads/vit_class_3_checkpoint.bin", map_location='cpu') # GPU 환경이 아닌 경우 'cpu'를 사용합니다.
vit.load_state_dict(checkpoint)

<All keys matched successfully>

In [20]:
# 이미지 전처리 함수 정의
from torchvision import transforms
from PIL import Image

preprocess = transforms.Compose([
    transforms.Resize(224),  # 모델에 맞는 크기로 조정
    transforms.CenterCrop(224),  # 중앙을 기준으로 자르기
    transforms.ToTensor(),  # Tensor로 변환
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # 정규화
])

# 이미지 불러오기 및 전처리
image_path = "C:/Users/chldy/Downloads/aa.jpg"
input_image = Image.open(image_path)
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # 배치 차원을 추가

In [ ]:
# GPU가 있으면 GPU로 이동
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
input_batch = input_batch.to(device)

# 예측 수행
with torch.no_grad():
    output = model(input_batch)

# 출력 결과 처리 (예: 소프트맥스를 사용한 확률 변환)
probabilities = torch.nn.functional.softmax(output[0], dim=0)

# 결과 확인 (클래스 레이블이 있는 경우)
# 예를 들어, ImageNet 데이터셋의 레이블 파일을 사용하는 경우:
# labels_map = json.load(open("path_to_labels_map.json"))
# labels_map = [labels_map[str(i)] for i in range(1000)]

# top-5 예측 결과 출력
top5_prob, top5_catid = torch.topk(probabilities, 3)
for i in range(top5_prob.size(0)):
    print(f"{i+1}: {top5_prob[i].item()*100:.2f}% - Class ID: {top5_catid[i].item()}")
    # 클래스 레이블이 있다면:
    # print(f"{i+1}: {top5_prob[i].item()*100:.2f}% - {labels_map[top5_catid[i].item()]}")


In [21]:
import torch.nn.functional as F


with torch.no_grad():
    # MLP-Mixer 예측
    mlp_output = mixer(input_batch)
    mlp_probs = F.softmax(mlp_output, dim=1)
    
    # ViT 예측
    vit_output = vit(input_batch)
    vit_probs = F.softmax(vit_output, dim=1)
    
    # 평균 앙상블
    avg_probs = (mlp_probs + vit_probs) / 2
    final_prediction = avg_probs.argmax(dim=1).item()

# 각각의 모델 예측 확률 및 최종 예측 확률 출력
print("MLP-Mixer probabilities:", mlp_probs.numpy())
print("ViT probabilities:", vit_probs.numpy())
print("Ensemble probabilities:", avg_probs.numpy())
print("Final prediction:", final_prediction)

MLP-Mixer probabilities: [[0.33917397 0.34182787 0.31899816]]
ViT probabilities: [[9.9911350e-01 8.5955817e-04 2.6956297e-05]]
Ensemble probabilities: [[0.66914374 0.17134371 0.15951256]]
Final prediction: 0
